Get some transactions
Add plaid merchant, add website

based on https://github.com/meetcleo/data-science/blob/master/transaction-enrichment/notebooks/transactions_categories/T1-24/Data_for_TM_annotations.ipynb



In [1]:
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
from cleodata.utils.secrets import get_secret
from cleodata.sources.sync.sync import SyncDataSource
boto3.setup_default_session(profile_name='DataScientist-878877078763')
redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

Attempting to uninstrument while already uninstrumented


2024-05-25 07:18:39 [debug    ] fetching credentials          
2024-05-25 07:18:41 [info     ] Credentials acquired          
2024-05-25 07:18:41 [info     ] Built connection pool         


In [2]:
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import s3fs
from fastparquet import ParquetFile

In [5]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")

def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3


 - **Description 1**: Text description of transactions: Internal description or processed and cleared description from external provider 

 - **Description 2**: Text description of transactions, original from external provider. The text returned by the financial institution to describe the transaction. 

 - **Additional Merchant name (original):** Merchant name of transaction from external provider 

 - **Original Transfer counterparty name:** Extracted from some transaction information from external provider 

 - **Location:** If not empty, will include information as in following example:

{'lat': 28.607128, 'lon': -81.387253, 'city': 'Orlando', 'region': 'FL', 'address': '609 Lee Rd', 'country': None, 'postal_code': '32810', 'store_number': None}

- **Payment channel:** The channel used to make a payment. Possible values: online, in store, other: - online: transactions that took place online. - in store: transactions that were made at a physical location. - other: transactions that relate to banks, e.g. fees or deposits. 

- **Payee:** name of transfer receiver -+ Payer: name of transfer sender

- Google search link for “Additional Merchant name (original)”

- Google search link for “Description 1”

- Google search link for “Description 2”

- Google search link for “Original Transfer counterparty name”

- Google search link for “Description 2 and Original Transfer counterparty name”


In [6]:
start_date_s = '2024-05-13'
end_date_s = '2024-05-19'
date_range = pd.date_range(start=start_date_s, end=end_date_s)
# Convert the date range to a list of strings
date_list = date_range.strftime('%Y-%m-%d').tolist()
date_list

['2024-05-13',
 '2024-05-14',
 '2024-05-15',
 '2024-05-16',
 '2024-05-17',
 '2024-05-18',
 '2024-05-19']

In [7]:
start_date = date_list[0]
end_date = date_list[0]
print(start_date, end_date)

2024-05-13 2024-05-13


In [9]:
#list the files that start with that pattern
base_path = f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}"

list_s3_flies(base_path)

['cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-13_2024-05-13_240524_172213029_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-13_2024-05-13_240524_173712824_part_00.parquet']

In [13]:
dataset_id = "trans_2024-05-13_2024-05-13_240524_173712824_part_00"
path_file = f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}_{dataset_id}"
print(path_file)
df_trans = read_from_s3(path_file)


s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-13_2024-05-13_trans_2024-05-13_2024-05-13_240524_173712824_part_00
Nothing found
paths from a[]


In [47]:
run_one_off = 1

for one_date in date_list:
    start_date = one_date
    end_date = one_date
    if run_one_off==0 :
        df_trans_orig = redshift_source.fetch_data(sql_fct_trans_with_merchant)
    elif run_one_off == 1:
        dataset_id = datetime.now().strftime("%y%m%d_%H%M")
        sql_fct_trans_with_merchant = get_query(start_date, end_date)
        sql = f"""
        unload ('select * from ({sql_fct_trans_with_merchant.replace("'", "''")}
        )')
            to 's3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}_{dataset_id}'
            iam_role 'arn:aws:iam::878877078763:role/CleoRedshiftToS3'
            format as parquet
            cleanpath;"""
        print(sql)
        redshift_source.execute_query(sql)
        
        path_file = f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_{start_date}_{end_date}_{dataset_id}"
        print(path_file)
        #df_trans = read_from_s3(path_file)

        # bucket_name = path_file.split('://')[1].split('/')[0]
        # key = path_file.split('://')[1].split(bucket_name)[1][1:]
        # print(f"bucket {bucket_name}")
        # print(f"key {key}")
        # df_trans  = read_csv_s3(bucket_name, key )
    elif run_one_off ==2:
        path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-20_2024-05-20"
        df_trans = read_from_s3(path_file)


        unload ('select * from (  
  select ftt.transaction_id,
        ftt.corrected_made_on,
        ftt.amount,
        ftt.currency_code,
        ftt.description,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''original_description'')
          as original_description_plaid,
        ftt.merchant_name,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''merchant_name'') as merchant_name_plaid,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''transaction_type'')
          as transaction_type_plaid,
        json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''name'') AS counterparty_name,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''type'') AS counterparty_type,
      json_extract_path_text(ftt.login_provider_additional_attributes, ''counterparties'', ''0'', ''website'') AS counterparty_website,
      json_extract_path

In [53]:
path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/raw/trans_2024-05-17_2024-05-17_240524_1743"

In [54]:
df_trans = read_from_s3(path_file)
df_trans.shape

Nothing found
paths from a[]


AttributeError: 'NoneType' object has no attribute 'shape'

In [50]:
df_trans

In [35]:
path_file_processed = path_file.split('raw/')[0]+'processed/'+path_file.split('raw/')[1]
path_file_processed


's3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/trans_2024-05-14_2024-05-14_240524_1725'

In [36]:
df_trans

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,category_primary_plaid,payment_channel,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes
0,9881624483,2024-05-14,38.01,USD,APPLE CASH INS 05/14 #XXXXX6387 PMNT RCVD APPL...,APPLE CASH INS 05/14 #XXXXX6387 PMNT RCVD APPL...,None,,special,Apple Cash,...,TRANSFER_IN,other,,,Cupertino,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
1,9881624477,2024-05-14,40.38,USD,APPLE CASH INS 05/14 #XXXXX3815 PMNT RCVD APPL...,APPLE CASH INS 05/14 #XXXXX3815 PMNT RCVD APPL...,None,,special,Apple Cash,...,TRANSFER_IN,other,,,Cupertino,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
2,9881624479,2024-05-14,30.74,USD,APPLE CASH INS 05/14 #XXXXX5173 PMNT RCVD APPL...,APPLE CASH INS 05/14 #XXXXX5173 PMNT RCVD APPL...,None,,special,Apple Cash,...,TRANSFER_IN,other,,,Cupertino,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
3,9881624486,2024-05-14,22.66,USD,APPLE CASH INS 05/14 #XXXXX9249 PMNT RCVD APPL...,APPLE CASH INS 05/14 #XXXXX9249 PMNT RCVD APPL...,None,,special,Apple Cash,...,TRANSFER_IN,other,,,Cupertino,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."
4,9881624484,2024-05-14,26.60,USD,APPLE CASH INS 05/14 #XXXXX9077 PMNT RCVD APPL...,APPLE CASH INS 05/14 #XXXXX9077 PMNT RCVD APPL...,None,,special,Apple Cash,...,TRANSFER_IN,other,,,Cupertino,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""..."


If merchant name is not present, user merchant_name_plaid. 
If merchant_name is not present, use counterparty_name



In [38]:
import re
import numpy as np

In [39]:
# Coalescing mercant names

#replace None with null
#replace Nan
# Replace empty spaces, None, and strings with only spaces with NaN
df_trans['merchant_name'] = df_trans['merchant_name'].replace(r'^\s*$', np.nan, regex=True)
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].replace(r'^\s*$', np.nan, regex=True)


df_trans['merchant_name'] = df_trans['merchant_name'].map({'None':None})
df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].map({'':None,' ':None})
# create a column combined_merchant where we take the any merchant name : Cleo, or plaid, or counterparty 
df_trans['merchant_name_combined'] = df_trans['merchant_name'].combine_first(df_trans['merchant_name_plaid'])
df_trans['merchant_name_combined'] = df_trans['merchant_name_combined'].combine_first(df_trans['counterparty_name'])
# Remove data without merchant name for training data
df_trans_cln = df_trans[(~df_trans['merchant_name_combined'].isnull()) & ~df_trans['merchant_name_combined'].isin(['',' '])]

# Coalescing descriptions 
# if original_description_plaid is empty use description
df_trans['description_combined'] = df_trans['original_description_plaid'].combine_first(df_trans['description'])
df_trans['len_description'] = df_trans['description_combined'].apply(lambda x: len(x))
df_trans = df_trans[df_trans['len_description'] >2]
##
#replace 'other' with ''
df_trans['payment_channel_processed'] = df_trans['payment_channel'].apply(lambda x: 'None' if x == 'other' else x)
# do some light processing to make strings shorter
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid'].apply(lambda x: re.sub('\\\\+','\\\\',x))
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(r'\d{4,}', ' ', x))

# #if there are 4 or more integers replace with a space
# output_string = re.sub(r'\d{4,}', ' ', df_data_raw['sentence'][1000050])
df_trans['original_description_plaid_processed'] =  df_trans['original_description_plaid_processed'].apply(lambda x: re.sub(' +',' ',x))
df_trans.reset_index(drop=True, inplace=True)

/var/folders/96/34ycxb693ss1n858z5ck1xwh0000gn/T/ipykernel_38432/2131297994.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_trans['merchant_name_plaid'] = df_trans['merchant_name_plaid'].replace(r'^\s*$', np.nan, regex=True)


In [ ]:
# There are some cases where Chime is the merchant but it isn't mentioned in the description, so remove these
df_trans['Chime in descr'] = df_trans['original_description_plaid'].apply(lambda x: 'Chime' in x)
df_trans = df_trans_cln[(df_trans['Chime in descr'] & (df_trans['merchant_name_combined']=='Chime')) | (df_trans['merchant_name_combined']!='Chime')]
df_trans.reset_index(drop=True, inplace=True)
df_trans

In [40]:
df_trans_no_merchant = df_trans[(df_trans['merchant_name_combined'].isnull()) | df_trans['merchant_name_combined'].isin(['',' '])]

In [41]:
df_trans_no_merchant

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,description_combined,len_description,payment_channel_processed,original_description_plaid_processed


In [ ]:
print(df_trans.shape)
print(df_trans_cln.shape)

(6492894, 29)
(5336831, 29)


In [ ]:
df_trans_cln['merchant_name_combined'].value_counts()

merchant_name_combined
Cash App                   346486
Zelle                      172242
Walmart                    115901
Apple                      114375
Earnin                     108338
                            ...  
Old River Donuts                1
Burnside                        1
Mjp Management                  1
Spotifyusai Angel Lopez         1
Anju House                      1
Name: count, Length: 426343, dtype: int64

In [ ]:
nunique_merchants = df_trans_cln['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

There are 426343  in the dataset


/var/folders/96/34ycxb693ss1n858z5ck1xwh0000gn/T/ipykernel_25025/3417653004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_cln['Chime in descr'] = df_trans_cln['original_description_plaid'].apply(lambda x: 'Chime' in x)


,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
0,9848923081,2024-05-20,25.00,USD,ZELLE P2P PAYMENT RECEIVED ON 05/19 \,ZELLE P2P PAYMENT RECEIVED ON 05/19 \\,NaN,None,special,Zelle,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Zelle,False
1,9848926035,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",NaN,NaN,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
2,9848926034,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",NaN,NaN,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
3,9848945525,2024-05-20,19.65,USD,Transfer from Cash App,Transfer from Cash App,NaN,None,special,Cash App,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Cash App,False
4,9848948208,2024-05-20,-100.00,USD,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,NaN,NaN,place,Q Nails,...,,,Newport,TN,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Q Nails,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291196,9857334046,2024-05-20,-15.00,USD,FanDuel,FanDuel,NaN,NaN,place,FanDuel,...,,,,,,None,,"{""datetime"": ""2024-05-20T20:09:51.000Z"", ""loca...",FanDuel,False
5291197,9857334047,2024-05-20,-25.00,USD,FanDuel,FanDuel,NaN,NaN,place,FanDuel,...,,,,,,None,,"{""datetime"": ""2024-05-20T20:09:52.000Z"", ""loca...",FanDuel,False
5291198,9857334048,2024-05-20,-25.00,USD,FanDuel,FanDuel,NaN,NaN,place,FanDuel,...,,,,,,None,,"{""datetime"": ""2024-05-20T20:09:51.000Z"", ""loca...",FanDuel,False
5291199,9857334049,2024-05-20,-25.00,USD,FanDuel,FanDuel,NaN,NaN,place,FanDuel,...,,,,,,None,,"{""datetime"": ""2024-05-20T20:09:51.000Z"", ""loca...",FanDuel,False


In [ ]:
df_counts = df_trans_cln2['merchant_name_combined'].value_counts()
df_counts
df_counts[0:30]

merchant_name_combined
Cash App              346486
Zelle                 172242
Walmart               115901
Apple                 114375
Earnin                108338
McDonald''s            96105
Cleo                   94816
Amazon                 90189
Apple Cash             82400
MoneyLion              68428
Uber                   63681
Varo                   61104
Dave                   60827
Brigit                 57196
Chime                  56630
Venmo                  56618
Empower                53624
Wells Fargo            47905
Albert                 47596
Shell                  44768
AfterPay               44643
PayPal                 44308
7-Eleven               43490
Dollar General         42236
FanDuel Sportsbook     41727
Circle K               41000
USAA                   40737
Uber Eats              38169
Klover App             37711
Starbucks              34059
Name: count, dtype: int64

In [ ]:
nunique_merchants = df_trans_cln['merchant_name_combined'].nunique()
print(f"There are {nunique_merchants}  in the dataset")

There are 426343  in the dataset


In [ ]:
df_trans_cln2[df_trans_cln2['merchant_name_combined']=="Uber Eats"]

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
248,9849310484,2024-05-20,-11.53,USD,Uber Eats,"DB DEBIT - 5-19-2024, UBER EATS ...",NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
270,9849334721,2024-05-20,-21.50,USD,Uber Eats,WITHDRAWAL. POS 0519 1945 819414 UBER *EATS ...,NaN,NaN,place,Uber Eats,...,,,San Francisco,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
491,9849685081,2024-05-20,-22.72,USD,Uber Eats,DEBIT CARD DEBIT / ref #000872020321 05-18-202...,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
760,9849846630,2024-05-20,-16.98,USD,Uber Eats,Uber Eats,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
761,9849846631,2024-05-20,-45.10,USD,Uber Eats,Uber Eats,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291100,9869969452,2024-05-20,-41.30,USD,Uber Eats,UBER *EATS SAN FRANCISCO CA 816488 05/19,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
5291110,9869969450,2024-05-20,34.62,USD,Uber Eats,UBER *EATS SAN FRANCISCO CA 230274 05/19,NaN,NaN,place,Uber Eats,...,,,San Francisco,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
5291150,9869976693,2024-05-20,-7.92,USD,Uber Eats,UBER *EATS SAN FRANCISCO CA 585375 05/20,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False
5291151,9869976695,2024-05-20,-33.56,USD,Uber Eats,UBER *EATS SAN FRANCISCO CA 054331 05/20,NaN,NaN,place,Uber Eats,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Uber Eats,False


In [ ]:
# examples where the description and the merchant name are the same are probably not too informative
df_trans_cln3 = df_trans_cln2[df_trans_cln2['merchant_name_combined']!=df_trans_cln2['original_description_plaid']]
print(df_trans_cln3.shape)
df_trans_cln3['merchant_name_combined'].value_counts()[:-40]

(4930071, 30)


merchant_name_combined
Cash App                     341036
Zelle                        172238
Earnin                       107179
Apple                        103460
Walmart                       97783
                              ...  
Element By Marriot                1
Bottle House-dc                   1
The Barac Co                      1
Antaniah White                    1
Animal Dermatology Clinic         1
Name: count, Length: 385024, dtype: int64

In [ ]:
def look_up_for_one_merchant(merchant_name, df):
    return df[df['merchant_name_combined']==merchant_name]

In [25]:
look_up_for_one_merchant('Affirm', df_trans_cln3)

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
958,9849972534,2024-05-20,18.07,USD,AFFIRM *PAYMENT,AFFIRM *PAYMENT 855-423-3729 CA,NaN,NaN,place,Affirm,...,,,,CA,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
2092,9850419943,2024-05-20,-7.13,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
2093,9850419944,2024-05-20,-12.72,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
2094,9850419945,2024-05-20,-13.95,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
2095,9850419946,2024-05-20,-13.95,USD,AFFIRM INC AFFIRM PAY,AFFIRM INC AFFIRM PAY,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288047,9869632565,2024-05-20,-512.10,USD,ACH Debit - Affirm Inc Affirm Pay 2100002,ACH Debit - Affirm Inc Affirm Pay 2100002,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
5289254,9869748085,2024-05-20,-15.33,USD,AFFIRM INC AFFIRM PAY 9381277 WEB ID: 0000317218,AFFIRM INC AFFIRM PAY 9381277 WEB ID: 0000317218,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
5290784,9869937536,2024-05-20,-7.51,USD,AFFIRM * PAY N628ZC3O CA 05/19,AFFIRM * PAY N628ZC3O 855-423-3729 CA 05/19,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False
5290795,9869937538,2024-05-20,-12.96,USD,AFFIRM * PAY OVXOUFVC CA 05/19,AFFIRM * PAY OVXOUFVC 855-423-3729 CA 05/19,NaN,NaN,place,Affirm,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Affirm,False


In [ ]:
df_trans_cln3.columns

Index(['transaction_id', 'corrected_made_on', 'amount', 'currency_code',
       'description', 'original_description_plaid', 'merchant_name',
       'merchant_name_plaid', 'transaction_type_plaid', 'counterparty_name',
       'counterparty_type', 'counterparty_website', 'counterparty_logo_url',
       'counterparty_entity_id', 'counterparty_confidence_level',
       'merchant_id', 'company', 'category_detailed_plaid',
       'category_primary_plaid', 'payment_channel', 'payment_method',
       'payment_processor_method', 'city', 'region', 'country', 'status',
       'decline_code', 'login_provider_additional_attributes',
       'merchant_name_combined', 'Chime in descr'],
      dtype='object')

In [ ]:
columns_to_keep = ['transaction_id','corrected_made_on','amount','original_description_plaid','merchant_name_combined','payment_channel']

In [ ]:
df_trans_cln3[columns_to_keep]

,transaction_id,corrected_made_on,amount,original_description_plaid,merchant_name_combined,payment_channel
0,9848923081,2024-05-20,25.00,ZELLE P2P PAYMENT RECEIVED ON 05/19 \\,Zelle,other
1,9848926035,2024-05-20,-7.86,"Purchase / McDonalds 35052 142-3328095, TN (4633)",McDonald''s,in store
2,9848926034,2024-05-20,-7.86,"Purchase / McDonalds 35052 142-3328095, TN (4633)",McDonald''s,in store
3,9848945525,2024-05-20,19.65,Transfer from Cash App,Cash App,other
4,9848948208,2024-05-20,-100.00,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,Q Nails,in store
...,...,...,...,...,...,...
5291189,9869983888,2024-05-20,-2.17,PIZZARONI BELL GARDEN BELL GARDENS CA 05/19,Pizzaroni Bell Gardens,in store
5291190,9869983879,2024-05-20,-6.00,ALBERT CASH 000000249530974 WEB ID: 1475215705,Albert,other
5291191,9869983880,2024-05-20,-115.47,ANJU HOUSE LOS ANGELES CA 05/17,Anju House,in store
5291192,9869983882,2024-05-20,-130.00,SQ *FRANK ''N HANK''S BA Los Angeles CA 05/18,Frank N Hanks,in store


In [ ]:
# save this dataset and depending on the type of algorithm you can choose how to reorder the data

df_trans_cln3.to_parquet(path_file_processed)

In [ ]:
print(f"Finsihed writing file {path_file_processed}")

In [31]:
#df_trans_cln3.to_csv('/Users/claracastellanos/Documents/DATA/MERCHANTS/2024_05_20.csv')

In [104]:
#load data back in to make sure everythin is ok
df_tmp = read_from_s3("s3://cleo-data-science/transaction_enrichment/experimental_data/caste/biencoder_test1/data1")

In [105]:
df_tmp.shape

(956812, 30)

In [106]:
df_tmp

,transaction_id,corrected_made_on,amount,currency_code,description,original_description_plaid,merchant_name,merchant_name_plaid,transaction_type_plaid,counterparty_name,...,payment_method,payment_processor_method,city,region,country,status,decline_code,login_provider_additional_attributes,merchant_name_combined,Chime in descr
index,,,,,,,,,,,,,,,,,,,,,
0,9848923081,2024-05-20,25.00,USD,ZELLE P2P PAYMENT RECEIVED ON 05/19 \,ZELLE P2P PAYMENT RECEIVED ON 05/19 \\,None,None,special,Zelle,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Zelle,False
1,9848926035,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
2,9848926034,2024-05-20,-7.86,USD,McDonald''s,"Purchase / McDonalds 35052 142-3328095, TN (4633)",None,None,place,McDonald''s,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",McDonald''s,False
3,9848945525,2024-05-20,19.65,USD,Transfer from Cash App,Transfer from Cash App,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Cash App,False
4,9848948208,2024-05-20,-100.00,USD,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,POS Withdrawal (FIS) Q NAILS Q NAILS NEWPORT T...,None,None,place,Q Nails,...,,,Newport,TN,,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Q Nails,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170552,9857528533,2024-05-20,-63.27,USD,Cash App*Anthony Cenic,Cash App*Anthony Cenic,None,None,special,Cash App,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:38:03.000Z"", ""loca...",Cash App,False
1170554,9857528559,2024-05-20,-34.70,USD,Point Of Sale Withdrawal 445488848992 DD DOORD...,Point Of Sale Withdrawal 445488848992 DD DOORD...,None,None,place,Mamichuyt,...,,,,CA,US,None,,"{""datetime"": null, ""location"": {""lat"": null, ""...",Mamichuyt,False
1170555,9857528629,2024-05-20,-10.28,USD,Transfer to Chime Savings Account,Transfer to Chime Savings Account,None,None,special,Chime,...,,,,,,None,,"{""datetime"": ""2024-05-20T19:11:52.000Z"", ""loca...",Chime,True


In [ ]:
df_trans_cln3.to_parquet()